In [1]:
import os.path
import numpy as np

from pynextsim.nextsim_config import NextsimConfig

In [2]:
def create_grid_params(params, exporter_path_root):
    vectors = list(params.values())
    grids = np.meshgrid(*vectors, indexing='ij')
    all_vals = np.vstack([a.flatten() for a in grids]).T

    grid_params = {}
    for i, vals in enumerate(all_vals):
        grid_params[i] = {}
        for j, key in enumerate(params.keys()):
            grid_params[i][key] = all_vals[i,j]

    for i in grid_params:
        grid_params[i]['output.exporter_path'] = f'{exporter_path_root}/exp_{i:05d}'
    return grid_params

In [3]:
conf_dir = '/Home/antonk/n/nextsim-env/config_files'
conf_template_name = f'{conf_dir}/hrmny_sa05free_template.cfg'
sbatch_template_name = f'{conf_dir}/sbatch_nextsim_template.sh'
exporter_path_root = '/cluster/work/users/akorosov/harmony'
params = {
    'dynamics.compression_factor': [5e6, 10e6, 15e6],
    'dynamics.C_lab': [0.2e6, 0.4e6, 0.6e6, 1e6, 1.6e6, 2.5e6, 4e6],
}

grid_params = create_grid_params(params, exporter_path_root)

template = NextsimConfig(conf_template_name)

with open(sbatch_template_name) as fid:
    sbatch_template = fid.read()
    
print(template)
print(sbatch_template)

{'mesh': {'filename': 'small_arctic_5km.msh'}, 'setup': {'ice-type': 'topaz_osisaf_icesat', 'ocean-type': 'topaz', 'atmosphere-type': 'era5'}, 'simul': {'spinup_duration': '1', 'timestep': '450', 'time_init': '2006-11-15', 'duration': '100'}, 'thermo': {'h_young_max': '0.5'}, 'dynamics': {'time_relaxation_damage': '15', 'compression_factor': '10e3', 'C_lab': '2e6', 'substeps': '90'}, 'output': {'output_per_day': '4', 'exporter_path': 'None', 'export_after_regrid': 'true', 'export_before_regrid': 'true'}}
#!/bin/bash -x

## Project:
#SBATCH --account=nn9878k

## Job name:
#SBATCH --job-name=EXP_NAME

## Output file
#SBATCH --output=/cluster/home/akorosov/harmony.%j.log         # Stdout & stderr

## Wall time limit:
#SBATCH --time=1-12:00:00

## Number of nodes:
#SBATCH --nodes=1

## Number of tasks (total)
#SBATCH --ntasks=32

## Set OMP_NUM_THREADS
#SBATCH --cpus-per-task=1

## Queue Option (preproc for 1 node; normal for >4 nodes; also can be bigmem)
#

## Ensure we have a clean envir

In [4]:
for i in grid_params:
    for param in grid_params[i]:
        p0, p1 = param.split('.')
        template[p0][p1] = grid_params[i][param]
        conf_name = conf_template_name.replace('template.cfg', f'{i:05}.cfg')
        with open(conf_name, 'wt') as fid:
            template.write(fid)
            
        sbatch_name = sbatch_template_name.replace('template.sh', f'{i:05}.sh')
        with open(sbatch_name, 'wt') as fid:
            fid.write(sbatch_template.replace('EXP_NAME', os.path.splitext(os.path.basename(conf_name))[0]))